In [5]:
# %% [markdown]
# # 02 — Statistical Analysis of Query Performance
# 
# Acest notebook rulează analize statistice inferențiale după etapa EDA:
# 
# 1. **Încărcarea datelor** (aceleași CSV‑uri brute) și recrearea seturilor `merged_index` și `merged_shards`.
# 2. **Teste de semnificație pereche**: *paired t‑test* și Wilcoxon.
# 3. **Mărimea efectului**: Cohen’s *d*.
# 4. **Intervale de încredere bootstrap 95 %** pentru Δ‑medie și %‑îmbunătățire.
# 5. **Analiză de putere** pentru dimensiunea eșantionului necesar.
# 6. **Salvare rezultate**: tabele CSV + grafic al bootstrap‑ului.
# 
# ---
# %%
# 1️ Setup – imports & paths
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
from statsmodels.stats.power import TTestPower

BASE_DIR = Path("D:/Disertatie/1.database_performance_analysis").resolve()
RAW_DIR  = BASE_DIR / "data" / "raw"
TABLES_DIR = BASE_DIR / "results" / "tables"
FIG_STAT_DIR = BASE_DIR / "results" / "figures" / "statistical"
FIG_STAT_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

RAW_FILES = {
    "no_index"    : RAW_DIR / "queries_no_index.csv",
    "with_index"  : RAW_DIR / "queries_with_index.csv",
    "no_shards"   : RAW_DIR / "queries_no_shards.csv",
    "with_shards" : RAW_DIR / "queries_with_shards.csv",
}

# %%
# 2️ Load & merge datasets (same logic as EDA)
idx    = pd.read_csv(RAW_FILES["with_index"],   low_memory=False)
no     = pd.read_csv(RAW_FILES["no_index"],     low_memory=False)
sh_idx = pd.read_csv(RAW_FILES["with_shards"],  low_memory=False)
sh_no  = pd.read_csv(RAW_FILES["no_shards"],    low_memory=False)

for df in [idx, no, sh_idx, sh_no]:
    df["query_id"] = df["query_id"].astype(str)

base_cols = ["query_id", "documents_returned"]

idx2 = idx.rename(columns={"execution_time_ms": "exec_time_index"})
no2  = no .rename(columns={"execution_time_ms": "exec_time_no_index"})
merged_index = (
    idx2[base_cols + ["exec_time_index"]]
    .merge(no2[base_cols + ["exec_time_no_index"]], on="query_id", how="inner")
)
merged_index["diff_ms"] = merged_index["exec_time_no_index"] - merged_index["exec_time_index"]
merged_index["pct_impr"] = merged_index["diff_ms"] / merged_index["exec_time_no_index"] * 100

sh_idx2 = sh_idx.rename(columns={"execution_time_ms": "exec_time_shards"})
sh_no2  = sh_no .rename(columns={"execution_time_ms": "exec_time_no_shards"})
merged_shards = (
    sh_idx2[base_cols + ["exec_time_shards"]]
    .merge(sh_no2[base_cols + ["exec_time_no_shards"]], on="query_id", how="inner")
)
merged_shards["diff_ms"] = merged_shards["exec_time_no_shards"] - merged_shards["exec_time_shards"]
merged_shards["pct_impr"] = merged_shards["diff_ms"] / merged_shards["exec_time_no_shards"] * 100

# %%
# Helper: run paired tests, effect size, bootstrap CI

def paired_stats(df, col_before, col_after, label):
    x = df[col_before]
    y = df[col_after]
    diff = x - y  # positive => before slower than after (improvement)

    # Paired t‑test (H0: mean diff = 0)
    t_stat, p_t = stats.ttest_rel(x, y)

    # Wilcoxon signed‑rank
    w_stat, p_w = stats.wilcoxon(x, y)

    # Cohen's d (paired)
    d = diff.mean() / diff.std(ddof=1)

    # Bootstrap CI for mean diff & pct improvement
    rng = np.random.default_rng(42)
    boots = rng.choice(diff, size=(5000, len(diff)), replace=True).mean(axis=1)
    ci_low, ci_high = np.percentile(boots, [2.5, 97.5])

    # Power analysis (detect observed effect with alpha=0.05, power=0.8)
    power_calc = TTestPower()
    n_required = power_calc.solve_power(effect_size=abs(d), alpha=0.05, power=0.8, alternative='two-sided')

    summary = {
        "group": label,
        "n": len(diff),
        "mean_before": x.mean(),
        "mean_after": y.mean(),
        "mean_diff": diff.mean(),
        "cohen_d": d,
        "t_stat": t_stat, "p_t": p_t,
        "wilcoxon": w_stat, "p_w": p_w,
        "ci_low": ci_low, "ci_high": ci_high,
        "n_required_80p": n_required,
    }

    # Save bootstrap distribution plot
    plt.figure(figsize=(4,3))
    plt.hist(boots, bins=40, density=True)
    plt.axvline(ci_low, color='red', ls='--'); plt.axvline(ci_high, color='red', ls='--')
    plt.title(f"Bootstrap CI – {label}")
    plt.xlabel("Mean diff (ms)")
    plt.tight_layout()
    plt.savefig(FIG_STAT_DIR / f"bootstrap_{label}.png")
    plt.close()

    return summary

In [6]:
# %%
# 3️ Run analyses
summary_index  = paired_stats(merged_index,  "exec_time_no_index",  "exec_time_index",  "index")
summary_shards = paired_stats(merged_shards, "exec_time_no_shards", "exec_time_shards", "shards")

summary_df = pd.DataFrame([summary_index, summary_shards])
summary_df.to_csv(TABLES_DIR / "statistical_summary.csv", index=False)
print(summary_df.round(3))

    group    n  mean_before  mean_after  mean_diff  cohen_d  t_stat    p_t  \
0   index  122       66.131      65.311      0.820    0.031   0.344  0.732   
1  shards  122       56.057      66.131    -10.074   -0.214  -2.360  0.020   

   wilcoxon    p_w  ci_low  ci_high  n_required_80p  
0    2972.5  0.477  -3.426    5.697        8106.119  
1     349.0  0.000 -16.131   -0.549         173.837  


In [7]:
# %%
# 4️ Save per-query diff table for deep‑dive
merged_index.to_csv(TABLES_DIR / "diff_per_query_index.csv", index=False)
merged_shards.to_csv(TABLES_DIR / "diff_per_query_shards.csv", index=False)

print("\nAll statistical outputs saved:\n",
      " • statistical_summary.csv\n",
      " • diff_per_query_*.csv\n",
      " • bootstrap_*.png (in figures/statistical)")


All statistical outputs saved:
  • statistical_summary.csv
  • diff_per_query_*.csv
  • bootstrap_*.png (in figures/statistical)


In [8]:
# %%
# 5️ Basic sanity check
assert summary_df.shape[0] == 2, "Expected two summary rows!"
print("Notebook finished ")

Notebook finished 
